In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import json 
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [4]:
with open('/content/drive/MyDrive/codefest/intents.json') as file:
    data = json.load(file)
    
training_sentences = []
training_labels = []
labels = []
responses = []


for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
num_classes = len(labels)

In [5]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

In [6]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [7]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 44)                748       
                                                                 
Total params: 17,292
Trainable params: 17,292
Non-trainable params: 0
____________________________________________________

In [8]:
epochs = 1000
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Epoch 1/1000
4/4 [==============================] - 1s 6ms/step - loss: 3.7842 - accuracy: 0.0579
Epoch 2/1000
4/4 [==============================] - 0s 4ms/step - loss: 3.7820 - accuracy: 0.0413
Epoch 3/1000
4/4 [==============================] - 0s 4ms/step - loss: 3.7799 - accuracy: 0.0413
Epoch 4/1000
4/4 [==============================] - 0s 5ms/step - loss: 3.7780 - accuracy: 0.0413
Epoch 5/1000
4/4 [==============================] - 0s 4ms/step - loss: 3.7761 - accuracy: 0.0413
Epoch 6/1000
4/4 [==============================] - 0s 3ms/step - loss: 3.7740 - accuracy: 0.0413
Epoch 7/1000
4/4 [==============================] - 0s 4ms/step - loss: 3.7717 - accuracy: 0.0413
Epoch 8/1000
4/4 [==============================] - 0s 4ms/step - loss: 3.7693 - accuracy: 0.0413
Epoch 9/1000
4/4 [==============================] - 0s 5ms/step - loss: 3.7665 - accuracy: 0.0413
Epoch 10/1000
4/4 [==============================] - 0s 5ms/step - loss: 3.7636 - accuracy: 0.0413
Epoch 11/1000
4/4 [

In [9]:
# to save the trained model
model.save("/content/drive/MyDrive/codefest/chat_model")

import pickle

# to save the fitted tokenizer
with open('/content/drive/MyDrive/codefest/tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# to save the fitted label encoder
with open('/content/drive/MyDrive/codefest/label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

INFO:tensorflow:Assets written to: /content/drive/MyDrive/codefest/chat_model/assets


In [14]:
import json 
import numpy as np
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder

import random
import pickle

with open("intents.json") as file:
    data = json.load(file)


def chat():
    # load trained model
    model = keras.models.load_model('/content/drive/MyDrive/codefest/chat_model')

    # load tokenizer object
    with open('/content/drive/MyDrive/codefest/tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

    # load label encoder object
    with open('/content/drive/MyDrive/codefest/label_encoder.pickle', 'rb') as enc:
        lbl_encoder = pickle.load(enc)

    # parameters
    max_len = 20
    
    inp = 'hello'

    result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                          truncating='post', maxlen=max_len))
    tag = lbl_encoder.inverse_transform([np.argmax(result)])

    for i in data['intents']:
        if i['tag'] == tag:
            result = np.random.choice(i['responses'])
            return result

chat()

'Hi there, how can I help?'

In [15]:
!pip install flask
!pip install flask-ngrok

In [66]:
import pandas as pd
from flask_ngrok import run_with_ngrok
from flask import request, jsonify, Flask
import random as rk

In [67]:
app = Flask(__name__) #the name of the application package
run_with_ngrok(app)

In [68]:
d = {
    "answer": "empty"
    }

In [69]:
@app.route("/") 

def home():
  '''
  The entire line below must be written in a single line.
  '''
  return "<marquee><h3> TO CHECK IN PUT ADD '/input' TO THE URL AND TO CHECK OUT PUT ADD '/output' TO THE URL.</h3></marquee>"


In [70]:
@app.route("/input") 

def input():
  return jsonify(d) # "d" is the dictionary we defined

@app.route('/product/<name>')
def get_product(name):
  model = keras.models.load_model('/content/drive/MyDrive/codefest/chat_model')

  # load tokenizer object
  with open('/content/drive/MyDrive/codefest/tokenizer.pickle', 'rb') as handle:
      tokenizer = pickle.load(handle)

  # load label encoder object
  with open('/content/drive/MyDrive/codefest/label_encoder.pickle', 'rb') as enc:
      lbl_encoder = pickle.load(enc)

  # parameters
  max_len = 20
  
  inp = name

  result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                        truncating='post', maxlen=max_len))
  tag = lbl_encoder.inverse_transform([np.argmax(result)])

  for i in data['intents']:
      if i['tag'] == tag:
          result = np.random.choice(i['responses'])
  d["answer"] = result
  response = jsonify(d)
  return jsonify(d)


In [71]:
@app.route('/output', methods=['GET','POST']) #output page
def predJson():
 pred = r.choice(["positive","negative"])
 nd = d # our input
 nd["prediction"]=pred
 return jsonify(nd)




In [ ]:
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://ad34-35-247-76-214.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [26/Dec/2021 08:31:54] "GET /product/hi HTTP/1.1" 200 -
127.0.0.1 - - [26/Dec/2021 08:31:54] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [26/Dec/2021 08:31:56] "GET /product/hi HTTP/1.1" 200 -
127.0.0.1 - - [26/Dec/2021 08:32:01] "GET /product/weather HTTP/1.1" 200 -
127.0.0.1 - - [26/Dec/2021 08:32:02] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [26/Dec/2021 08:32:05] "GET /product/weather HTTP/1.1" 200 -
127.0.0.1 - - [26/Dec/2021 08:32:15] "GET /product/ HTTP/1.1" 404 -
127.0.0.1 - - [26/Dec/2021 08:32:36] "GET /product/what%20is%20the%20weather HTTP/1.1" 200 -
127.0.0.1 - - [26/Dec/2021 08:32:57] "GET /product/Where%20is%20cabin HTTP/1.1" 200 -
